In [5]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
from langchain_teddynote import logging

logging.langsmith("CH03-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-Prompt


# Vanilla

In [7]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

In [8]:
# 모델
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4-turbo',
)

# 질의
question = '대한민국의 언어는 무엇입니까?'

# 답변
answer = llm.stream(question)
stream_response(answer)

대한민국의 공식 언어는 한국어입니다.

# FewShotPromptTemplate

- FewShot 학습을 위한 프롬프트 생성 
- 단일 텍스트 형식으로 예제를 제공하며, 이 형식을 기반으로 프롬프트 구성 
- 기본적으로 예제와 입력 데이터를 단순히 연결하는 방식으로 작동 
- 주로 질문-답변, 간단한 요약 등의 작업에서 활용 

In [9]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [10]:
examples = [
    {
        'question':'피카소와 모차르트 중 누가 더 어린 나이에 사망했나요?',
        'answer': """추가 질문이 필요한가요?: 예.
추가 질문: 피카소는 몇 살에 사망했나요?
중간 답변: 피카소는 91세에 사망했습니다.
추가 질문: 모차르트는 몇 살에 사망했나요?
중간 답변: 모차르트는 35세에 사망했습니다.
최종 답변: 모차르트 
""",
    },
    {
        'question':'페이스북의 창립자가 처음 만든 소셜 네트워크의 이름은 무엇인가요?',
        'answer': """추가 질문이 필요한가요?: 예.
추가 질문: 페이스북의 창립자는 누구인가요?
중간 답변: 페이스북의 창립자는 마크 저커버그 입니다. 
추가 질문: 마크 저커버그가 처음 만든 소셜 네트워크의 이름은 무엇인가요?
중간 답변: 마크 저커버그가 처음 만든 소셜 네트워크의 이름은 Facemash 입니다. 
최종 답변: Faacemash 
""",
    },
    {
        'question':'대한민국에서 지폐에 인물이 아닌 그림이 포함된 화폐가 있나요?',
        'answer': """추가 질문이 필요한가요?: 예.
추가 질문: 대한민국에서 발행된 지폐에 포함된 그림이 있나요?
중간 답변: 대한민국의 5만원건 지폐에 초충도가 포함되어 있습니다. 
추가 질문: 초충도는 인물이 아닌 그림인가요?
중간 답변: 초충도는 꽃과 곤충을 그린 그림입니다. 
최종 답변: 예
""",
    },
    {
        'question':'이순신 장군이 태어난 해의 명나라 황제는 누구인가요?',
        'answer': """추가 질문이 필요한가요?: 예.
추가 질문: 이순신 장군은 언제 태어났나요?
중간 답변: 이순신 장군은 1545년에 태어났습니다. 
추가 질문: 1545년에 명나라 황제는 누구입니까?
중간 답변: 1545년에 명나라의 황제는 가정제 입니다. 
최종 답변: 가정제 
""",
    },
    {
        'question':'태양계에서 지구보다 태양에 가까운 행성 중, 자전 속도가 공전 속도보다 느린 행성은 무엇인가요?',
        'answer': """추가 질문이 필요한가요?: 예.
추가 질문: 태양계에서 지구보다 태양에 가까운 행성은 어떤 행성인가요?
중간 답변: 태양계에서 지구보다 태양에 가까운 행성은 수성과 금성 입니다. 
추가 질문: 수성과 금성 중 자전 속도가 공전 속도 보다 느린 행성은 무엇인가요?
중간 답변: 금성은 자전 속도가 공전 속도 보다 느립니다. 
"최종 답변: 금성 
""",
    },
]

In [11]:
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question:
피카소와 모차르트 중 누가 더 어린 나이에 사망했나요?
Answer:
추가 질문이 필요한가요?: 예.
추가 질문: 피카소는 몇 살에 사망했나요?
중간 답변: 피카소는 91세에 사망했습니다.
추가 질문: 모차르트는 몇 살에 사망했나요?
중간 답변: 모차르트는 35세에 사망했습니다.
최종 답변: 모차르트 



In [12]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=['question']
)

question = 'NVIDIA 창립 연도에 젠슨황의 나이는 몇 살 인가요?'
prompt_final = prompt.format(question=question)
print(prompt_final)

Question:
피카소와 모차르트 중 누가 더 어린 나이에 사망했나요?
Answer:
추가 질문이 필요한가요?: 예.
추가 질문: 피카소는 몇 살에 사망했나요?
중간 답변: 피카소는 91세에 사망했습니다.
추가 질문: 모차르트는 몇 살에 사망했나요?
중간 답변: 모차르트는 35세에 사망했습니다.
최종 답변: 모차르트 


Question:
페이스북의 창립자가 처음 만든 소셜 네트워크의 이름은 무엇인가요?
Answer:
추가 질문이 필요한가요?: 예.
추가 질문: 페이스북의 창립자는 누구인가요?
중간 답변: 페이스북의 창립자는 마크 저커버그 입니다. 
추가 질문: 마크 저커버그가 처음 만든 소셜 네트워크의 이름은 무엇인가요?
중간 답변: 마크 저커버그가 처음 만든 소셜 네트워크의 이름은 Facemash 입니다. 
최종 답변: Faacemash 


Question:
대한민국에서 지폐에 인물이 아닌 그림이 포함된 화폐가 있나요?
Answer:
추가 질문이 필요한가요?: 예.
추가 질문: 대한민국에서 발행된 지폐에 포함된 그림이 있나요?
중간 답변: 대한민국의 5만원건 지폐에 초충도가 포함되어 있습니다. 
추가 질문: 초충도는 인물이 아닌 그림인가요?
중간 답변: 초충도는 꽃과 곤충을 그린 그림입니다. 
최종 답변: 예


Question:
이순신 장군이 태어난 해의 명나라 황제는 누구인가요?
Answer:
추가 질문이 필요한가요?: 예.
추가 질문: 이순신 장군은 언제 태어났나요?
중간 답변: 이순신 장군은 1545년에 태어났습니다. 
추가 질문: 1545년에 명나라 황제는 누구입니까?
중간 답변: 1545년에 명나라의 황제는 가정제 입니다. 
최종 답변: 가정제 


Question:
태양계에서 지구보다 태양에 가까운 행성 중, 자전 속도가 공전 속도보다 느린 행성은 무엇인가요?
Answer:
추가 질문이 필요한가요?: 예.
추가 질문: 태양계에서 지구보다 태양에 가까운 행성은 어떤 행성인가요?
중간 답변: 태양계에서 지구보다 태양에 가까운 행성은 수성과 금성

In [13]:
# 결과 출력
answer = llm.stream(prompt_final)
stream_response(answer)

추가 질문이 필요한가요?: 예.
추가 질문: NVIDIA는 언제 창립되었나요?
중간 답변: NVIDIA는 1993년에 창립되었습니다.
추가 질문: 젠슨 황은 언제 태어났나요?
중간 답변: 젠슨 황은 1963년에 태어났습니다.
최종 답변: 젠슨 황은 NVIDIA 창립 연도인 1993년에 30세였습니다.

In [14]:
# 코드 정리
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=['question']
)

chain = prompt | llm | StrOutputParser()

answer = chain.stream(
    {
        "question","NVIDIA 창립 연도에 젠슨황의 나이는 몇 살 인가요?"
    }
)
stream_response(answer)

추가 질문이 필요한가요?: 예.
추가 질문: NVIDIA는 언제 창립되었나요?
중간 답변: NVIDIA는 1993년에 창립되었습니다.
추가 질문: 젠슨 황은 언제 태어났나요?
중간 답변: 젠슨 황은 1963년에 태어났습니다.
최종 답변: 젠슨 황은 NVIDIA 창립 연도인 1993년에 30살이었습니다.

# Example Selector 

- 프롬프트에 맞는 에제는 선택해야 할 경우
- [Reference] https://api.python.langchain.com/en/latest/core/example_selectors.html

In [16]:
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector, SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [18]:
# 임베딩 초기화 
embeddings = OpenAIEmbeddings()

# FAISS DB
faiss_store = FAISS.from_texts(
    [example["question"] for example in examples],
    embeddings
)

# Example Selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    faiss_store,
    k=1, # 생성 예시 수
)

# 질문
question = 'NVIDIA 창립 연도에 젠슨황의 나이는 몇 살 인가요?'

# 입력과 가장 유사한 예시 선택 
selected_examples = example_selector.select_examples({'question':question})

# 결과 출력
print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

입력에 가장 유사한 예시:
NVIDIA 창립 연도에 젠슨황의 나이는 몇 살 인가요?

question:
이순신 장군이 태어난 해의 명나라 황제는 누구인가요?
answer:
추가 질문이 필요한가요?: 예.
추가 질문: 이순신 장군은 언제 태어났나요?
중간 답변: 이순신 장군은 1545년에 태어났습니다. 
추가 질문: 1545년에 명나라 황제는 누구입니까?
중간 답변: 1545년에 명나라의 황제는 가정제 입니다. 
최종 답변: 가정제 



In [20]:
# 프롬프트 템플릿 적용 
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix='Question:\n{question}\nAnswer:',
    input_variables=['question']
)

question = 'NVIDIA 창립 연도에 젠슨황의 나이는 몇 살 인가요?'
prompt_formatted = prompt.format(question=question)
print(prompt_formatted)

Question:
이순신 장군이 태어난 해의 명나라 황제는 누구인가요?
Answer:
추가 질문이 필요한가요?: 예.
추가 질문: 이순신 장군은 언제 태어났나요?
중간 답변: 이순신 장군은 1545년에 태어났습니다. 
추가 질문: 1545년에 명나라 황제는 누구입니까?
중간 답변: 1545년에 명나라의 황제는 가정제 입니다. 
최종 답변: 가정제 


Question:
NVIDIA 창립 연도에 젠슨황의 나이는 몇 살 인가요?
Answer:


In [21]:
# 템플릿 & 체인 구성 
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix='Question:\n{question}\nAnswer:',
    input_variables=['question']
)

chain = prompt | llm

answer = chain.stream(
    {'question':'NVIDIA 창립 연도에 젠슨황의 나이는 몇 살 인가요?'}
)
stream_response(answer)

추가 질문이 필요한가요?: 예.
추가 질문: NVIDIA는 언제 창립되었나요?
중간 답변: NVIDIA는 1993년에 창립되었습니다.
추가 질문: 젠슨황은 언제 태어났나요?
중간 답변: 젠슨황은 1963년에 태어났습니다.
최종 답변: 젠슨황은 NVIDIA 창립 연도인 1993년에 30살이었습니다.

# FewShotChatMessagePromptTemplate

- 대화형 메시지 기반으로 FewShot 학습을 위한 프롬프트 생성 
- 예제를 메시지 체인 형식으로 저장하며, 각각의 메시지는 역할과 내용을 명시 
- 역할(role)은 `system`, `human`,`ai`로 구분하여 설정
- 예제를 대화형 포맷으로 표현해야 하는 작업에 적합 

- 특징 비교

| **특징**               | **FewShotPromptTemplate**                                     | **FewShotChatMessagePromptTemplate**                        |
|------------------------|-------------------------------------------------------------|------------------------------------------------------------|
| **대상 모델**          | 일반 텍스트 기반 모델 (예: GPT-3)                            | 대화형 메시지 기반 모델 (예: ChatGPT)                       |
| **프롬프트 구조**      | 단일 텍스트 프롬프트                                         | 메시지 체인 구조                                           |
| **역할 지원**          | 지원하지 않음                                                | `system`, `user`, `assistant` 역할 지원                    |
| **출력 형식**          | 단일 텍스트 문자열                                           | 메시지 객체 리스트                                         |
| **사용 사례**          | 질문-답변, 요약 등 간단한 작업                               | 대화형 작업, 복잡한 대화 흐름 설정                        |
| **예제 표현 방식**     | 텍스트 형식으로 제공된 예제 사용                             | 메시지 역할과 내용을 포함한 대화형 예제 사용               |
| **유연성**             | 단순 작업에서 적합                                           | 대화 문맥과 역할 분리가 필요한 복잡한 작업에서 적합       |



In [24]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

In [25]:
examples = [
    {
        "instruction": "당신은 고객 서비스 평가 전문가 입니다. 다음 주어진 대화를 평가하고 개선점을 제안해 주세요",
        "input": """고객: 안녕하세요, 주문한 상품이 아직 도착하지 않았어요.
고객 서비스: 안녕하세요. 불편을 드려 죄송합니다. 주문 번호를 알려주시면 확인해 드리겠습니다.
고객: 12345입니다.
고객 서비스: 확인해보니 배송이 지연된 것으로 보입니다. 내일까지 도착할 예정입니다.
고객: 알겠습니다. 그런데 지연된 이유는 무엇인가요?
고객 서비스: 시스템 오류로 인해 출발이 늦어졌습니다. 불편을 끼쳐 드려 죄송합니다.""",
        "answer": """
고객 서비스 평가:
- 긍정적 측면:
  - 고객의 불만에 대해 사과하며 즉각적으로 문제 해결에 나선 점.
  - 고객의 주문 번호를 요청해 상황을 확인한 점.

- 개선점:
  - 지연 이유를 먼저 설명하여 고객의 의문을 미리 해소할 필요.
  - 문제 해결 이후 고객 만족도를 확인하거나 추가적인 지원이 필요한지 물어보는 단계가 부족.

- 개선된 응답 예시:
  고객 서비스: 안녕하세요. 불편을 드려 죄송합니다. 확인해본 결과, 시스템 오류로 인해 배송이 지연되었습니다. 불편을 끼쳐 드린 점 다시 한 번 사과드리며, 내일까지 상품이 도착할 예정입니다. 혹시 추가적인 도움이 필요하시다면 말씀해 주세요.
"""
    },
    {
        "instruction": "당신은 데이터 분석 전문가 입니다. 주어진 데이터의 주요 패턴을 분석해 주세요",
        "input": """데이터: 2023년 월별 매출 (단위: 백만 원)
1월: 100, 2월: 120, 3월: 140, 4월: 130, 5월: 150, 6월: 160, 7월: 170, 8월: 165, 9월: 175, 10월: 180, 11월: 200, 12월: 220""",
        "answer": """
분석 결과:
- **전반적인 추세**: 매출이 꾸준히 증가하는 양상을 보임.
- **상반기와 하반기 비교**:
  - 상반기 매출 총합: 800백만 원
  - 하반기 매출 총합: 1110백만 원
  - 하반기 매출이 상반기 대비 약 39% 증가.
- **특이점**: 11월과 12월의 급격한 매출 증가(각각 200, 220백만 원)가 두드러짐.
- **가능한 원인**:
  - 연말 소비 증가.
  - 특정 마케팅 캠페인 효과.
- **제안**:
  - 연말 효과를 활용한 추가 프로모션 전략 필요.
  - 상반기 매출 증가를 위해 하반기 성공 전략 분석 후 적용.
"""
    },
    {
        "instruction": "당신은 제품 리뷰 분석 전문가 입니다. 다음 주어진 리뷰를 분석하고 긍정적 및 부정적 요소를 요약해 주세요",
        "input": """리뷰: "이 제품은 정말 훌륭합니다. 배터리 지속 시간이 길고, 화면 품질도 뛰어납니다. 하지만 무게가 무겁고, 가격이 조금 비싼 것이 단점입니다." """,
        "answer": """
리뷰 분석 결과:
- **긍정적 요소**:
  - 배터리 지속 시간 우수.
  - 화면 품질 뛰어남.
- **부정적 요소**:
  - 무게가 무겁다.
  - 가격이 비싸다.
- **종합 의견**:
  - 제품은 전반적으로 성능 면에서 긍정적인 평가를 받았으나, 휴대성과 가격에 대한 개선이 필요함.
"""
    },
    {
        "instruction": "당신은 행사 기획 전문가 입니다. 주어진 정보를 바탕으로 행사 일정표를 작성해 주세요",
        "input": """행사명: 2024년 신제품 발표회
일시: 2024년 3월 15일
장소: 서울 코엑스 컨벤션 센터
참석 대상: 회사 임직원, 주요 고객, 언론 관계자
주요 내용: 신제품 발표, Q&A 세션, 네트워킹 이벤트""",
        "answer": """
행사 일정표: 2024년 신제품 발표회
- **일시**: 2024년 3월 15일
- **장소**: 서울 코엑스 컨벤션 센터
- **참석 대상**: 회사 임직원, 주요 고객, 언론 관계자

**일정**:
1. 10:00 - 10:30: 등록 및 환영 인사
2. 10:30 - 12:00: 신제품 발표
3. 12:00 - 13:30: 점심 및 네트워킹
4. 13:30 - 14:30: Q&A 세션
5. 14:30 - 15:00: 폐회 및 추가 네트워킹
"""
    },
    {
        "instruction": "당신은 논문 초록 작성 전문가 입니다. 주어진 내용을 바탕으로 논문의 초록을 작성해 주세요",
        "input": """논문 제목: 'AI를 활용한 재난 예측 시스템'
내용: 이 논문은 인공지능(AI)을 활용하여 자연재해를 예측하는 시스템을 제안한다. 시스템은 위성 데이터와 실시간 기상 정보를 결합하여 홍수, 태풍, 산불 등 다양한 자연재해를 조기에 감지하고 예측하는 데 초점을 맞춘다. 제안된 시스템은 기계 학습 알고리즘을 사용하여 과거 데이터를 분석하고, 정확도를 높이기 위해 최신 딥러닝 모델을 통합하였다. 실험 결과, 제안된 시스템은 기존 모델 대비 예측 정확도가 20% 향상되었음을 보여준다.""",
        "answer": """
초록:
본 논문은 인공지능(AI)을 활용하여 자연재해를 조기에 감지하고 예측하는 시스템을 제안한다. 제안된 시스템은 위성 데이터와 실시간 기상 정보를 결합하여 홍수, 태풍, 산불 등 다양한 재난 상황을 예측한다. 기계 학습 알고리즘과 최신 딥러닝 모델을 통합함으로써 예측 정확도를 개선하였다. 실험 결과, 본 시스템은 기존 모델 대비 20% 향상된 정확도를 보였으며, 이는 재난 대응 및 피해 최소화에 기여할 수 있음을 시사한다.
"""
    }
]

In [26]:
# 임베딩 초기화 
embeddings = OpenAIEmbeddings()

# FAISS 
faiss_store = FAISS.from_texts(
    [example["instruction"] + "\n" + example["input"] for example in examples],
    embeddings,
)

# 예제 프롬프트
example_prompt = ChatPromptTemplate.from_messages(
    [
        ('human','{instruction}:\n{input}'),
        ('ai', '{answer}'),
    ]
)

# Example Selector 
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, # 선택 가능한 예시 목록
    embeddings,
    faiss_store,
    k=1, # 생성할 예시 개수
)

# FewShot 프롬프트 템플릿 
fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)

In [27]:
question = {
    'instruction':'제안할 개선점을 작성해 주세요.',
    'input':'주문한 상품이 시스템 오류로 배송이 지연되고 있다.',
}

example_selector.select_examples(question)

[{'instruction': '당신은 고객 서비스 평가 전문가 입니다. 다음 주어진 대화를 평가하고 개선점을 제안해 주세요',
  'input': '고객: 안녕하세요, 주문한 상품이 아직 도착하지 않았어요.\n고객 서비스: 안녕하세요. 불편을 드려 죄송합니다. 주문 번호를 알려주시면 확인해 드리겠습니다.\n고객: 12345입니다.\n고객 서비스: 확인해보니 배송이 지연된 것으로 보입니다. 내일까지 도착할 예정입니다.\n고객: 알겠습니다. 그런데 지연된 이유는 무엇인가요?\n고객 서비스: 시스템 오류로 인해 출발이 늦어졌습니다. 불편을 끼쳐 드려 죄송합니다.',
  'answer': '\n고객 서비스 평가:\n- 긍정적 측면:\n  - 고객의 불만에 대해 사과하며 즉각적으로 문제 해결에 나선 점.\n  - 고객의 주문 번호를 요청해 상황을 확인한 점.\n\n- 개선점:\n  - 지연 이유를 먼저 설명하여 고객의 의문을 미리 해소할 필요.\n  - 문제 해결 이후 고객 만족도를 확인하거나 추가적인 지원이 필요한지 물어보는 단계가 부족.\n\n- 개선된 응답 예시:\n  고객 서비스: 안녕하세요. 불편을 드려 죄송합니다. 확인해본 결과, 시스템 오류로 인해 배송이 지연되었습니다. 불편을 끼쳐 드린 점 다시 한 번 사과드리며, 내일까지 상품이 도착할 예정입니다. 혹시 추가적인 도움이 필요하시다면 말씀해 주세요.\n'}]

In [28]:
prompt_final = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            'Your are a helpful assistant.'
        ),
        fewshot_prompt,
        ('human', '{instruction}\n{input}'),
    ]
)

In [29]:
chain = prompt_final | llm
answer = chain.stream(question)
stream_response(answer)

고객 서비스 개선점 제안:

1. **투명한 커뮤니케이션**: 배송 지연이 발생할 경우, 고객에게 먼저 연락하여 상황을 알리고, 지연의 원인과 예상 배송일을 명확히 전달해야 합니다. 이는 고객의 불안감을 줄이고 신뢰를 유지하는 데 도움이 됩니다.

2. **사과와 보상 제공**: 시스템 오류로 인한 불편에 대해 사과하는 것은 필수적입니다. 또한, 가능한 경우 작은 할인 쿠폰이나 다음 구매 시 사용할 수 있는 포인트를 제공하여 고객의 불편을 보상할 수 있습니다.

3. **지속적인 업데이트 제공**: 배송 상태에 변동이 있거나 추가 정보가 확인되면, 즉시 고객에게 업데이트를 제공해야 합니다. 이메일, 문자 메시지, 또는 고객이 선호하는 다른 방법을 통해 지속적으로 정보를 제공하는 것이 좋습니다.

4. **고객 지원 강화**: 배송 지연과 관련하여 고객이 문의할 수 있는 전용 핫라인이나 채팅 서비스를 마련하여, 고객이 언제든지 쉽게 연락할 수 있도록 합니다. 빠르고 정확한 응답을 제공하여 고객의 불편을 최소화합니다.

5. **시스템 개선**: 시스템 오류가 반복되지 않도록 내부 검토를 실시하고, 필요한 경우 시스템 업그레이드나 개선 작업을 진행합니다. 이는 장기적으로 비슷한 문제가 재발하는 것을 방지합니다.

이러한 개선점들을 통해 고객 만족도를 높이고, 비슷한 상황에서도 고객의 신뢰를 유지할 수 있습니다.

# 커스텀 유사도 계산 

- Example Selector로 유사도 계산시 instruction과 input이 필요함 
- 그러나 instruction만으로 검색하여 제대로된 유사도 검색 결과가 잘 나오지 않을 수 있음
- 이를 위해 커스텀 구현이 필요함 

In [32]:
from langchain_teddynote.prompts import CustomExampleSelector

In [44]:
custom_selector = CustomExampleSelector(
    examples,
    OpenAIEmbeddings()
)

custom_selector.select_examples({"instruction": "논문 초록을 작성해 주세요."})

[{'instruction': '당신은 논문 초록 작성 전문가 입니다. 주어진 내용을 바탕으로 논문의 초록을 작성해 주세요',
  'input': "논문 제목: 'AI를 활용한 재난 예측 시스템'\n내용: 이 논문은 인공지능(AI)을 활용하여 자연재해를 예측하는 시스템을 제안한다. 시스템은 위성 데이터와 실시간 기상 정보를 결합하여 홍수, 태풍, 산불 등 다양한 자연재해를 조기에 감지하고 예측하는 데 초점을 맞춘다. 제안된 시스템은 기계 학습 알고리즘을 사용하여 과거 데이터를 분석하고, 정확도를 높이기 위해 최신 딥러닝 모델을 통합하였다. 실험 결과, 제안된 시스템은 기존 모델 대비 예측 정확도가 20% 향상되었음을 보여준다.",
  'answer': '\n초록:\n본 논문은 인공지능(AI)을 활용하여 자연재해를 조기에 감지하고 예측하는 시스템을 제안한다. 제안된 시스템은 위성 데이터와 실시간 기상 정보를 결합하여 홍수, 태풍, 산불 등 다양한 재난 상황을 예측한다. 기계 학습 알고리즘과 최신 딥러닝 모델을 통합함으로써 예측 정확도를 개선하였다. 실험 결과, 본 시스템은 기존 모델 대비 20% 향상된 정확도를 보였으며, 이는 재난 대응 및 피해 최소화에 기여할 수 있음을 시사한다.\n'}]

In [34]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

custom_fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=custom_selector,  # 커스텀 예제 선택기 사용
    example_prompt=example_prompt,  # 예제 프롬프트 사용
)

custom_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        custom_fewshot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

In [35]:
chain = custom_prompt | llm
question = {
    'instruction':'제안할 개선점을 작성해 주세요.',
    'input':'주문한 상품이 시스템 오류로 배송이 지연되고 있다.',
}
stream_response(chain.stream(question))

주문한 상품의 배송 지연 문제를 해결하기 위한 개선점은 다음과 같습니다:

1. **시스템 안정성 강화**: 시스템 오류를 최소화하기 위해 소프트웨어 및 하드웨어의 안정성을 강화합니다. 정기적인 시스템 점검과 업데이트를 통해 예기치 않은 오류를 사전에 방지합니다.

2. **오류 감지 및 신속 대응 시스템 구축**: 시스템에 오류 감지 기능을 추가하여 문제가 발생하면 즉시 알림을 받고 신속하게 대응할 수 있도록 합니다. 이를 위해 IT 지원 팀의 대응 속도를 높이고, 24시간 모니터링 체계를 구축합니다.

3. **고객 서비스 강화**: 배송 지연이 발생했을 때 고객에게 즉시 알리고, 예상 배송일을 제공합니다. 또한, 고객의 불편을 최소화하기 위해 적절한 보상 방안을 마련합니다.

4. **배송 프로세스 최적화**: 배송 프로세스를 재검토하고, 효율성을 높이기 위한 방안을 모색합니다. 필요한 경우, 추가적인 물류 자원을 투입하여 배송 지연을 해결합니다.

5. **고객 피드백 시스템 강화**: 고객의 의견을 수집하고 이를 시스템 개선에 반영하여, 유사한 문제가 재발하지 않도록 합니다. 고객의 의견을 정기적으로 검토하고, 개선 사항을 신속하게 도입합니다.

이러한 개선점을 통해 시스템 오류로 인한 배송 지연 문제를 최소화하고, 고객 만족도를 높일 수 있습니다.

In [36]:
question = {
    "instruction": "문서를 요약해 주세요",
    "input": "이 문서는 '2023년 글로벌 경제 전망'에 관한 30페이지에 달하는 상세한 보고서입니다. 보고서는 세계 경제의 현재 상태, 주요 국가들의 경제 성장률, 글로벌 무역 동향, 그리고 다가오는 해에 대한 경제 예측을 다룹니다. 이 보고서는 또한 다양한 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 분석하고 있습니다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))

이 문서는 '2023년 글로벌 경제 전망'을 상세히 다룬 30페이지 보고서로, 세계 경제의 현재 상태와 주요 국가들의 경제 성장률, 글로벌 무역 동향을 분석하고, 다가오는 해의 경제 예측을 제공합니다. 또한, 경제적, 정치적, 환경적 요인들이 세계 경제에 미칠 영향을 평가하고 있습니다.

In [37]:
question = {
    "instruction": "문장을 교정해 주세요",
    "input": "회사는 올해 매출이 증가할 것으로 예상한다. 새로운 전략이 잘 작동하고 있다.",
}

# 실행 및 결과 출력
stream_response(chain.stream(question))

회사는 새로운 전략이 잘 작동하고 있어 올해 매출이 증가할 것으로 예상한다.

-----
** End of Documents **